# Credit Risk Resampling Techniques

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [96]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids 

In [6]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [60]:
# Create our features
X = df.copy()
X.drop('loan_status', axis=1, inplace=True)

# Create our target
y = df['loan_status']

In [61]:
X.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,own,52800,0.431818,5,1,22800
1,8400.0,6.692,own,43600,0.311927,3,0,13600
2,9000.0,6.963,rent,46100,0.349241,3,0,16100
3,10700.0,7.664,own,52700,0.430740,5,1,22700
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000


In [62]:
# Check the balance of our target values
Counter(y)

Counter({'low_risk': 75036, 'high_risk': 2500})

In [74]:
def label_encoding(data, feature):
    mapping_dict = {}
    for feat in feature:
        for i, feature_value in enumerate(list(data[feat].sort_values(ascending=False).unique())):
            mapping_dict[feature_value] = i+1
        data[feat] = data[feat].map(lambda x: mapping_dict[x])
    return data

In [75]:
X = label_encoding(X, ['homeowner'])

In [76]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [77]:
# Create the StandardScaler instance
ss = StandardScaler()

In [78]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
ss.fit(X_train)

StandardScaler()

In [79]:
# Scale the training and testing data
X_train_ss = ss.transform(X_train)
X_test_ss = ss.transform(X_test)

# Simple Logistic Regression

In [80]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_ss, y_train)

LogisticRegression(random_state=1)

In [81]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_ss)
balanced_accuracy_score(y_test, y_pred)

0.9854053094832482

In [82]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  586,    14],
       [  110, 18674]])

In [83]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.98      0.99      0.90      0.99      0.97       600
   low_risk       1.00      0.99      0.98      1.00      0.99      0.97     18784

avg / total       0.99      0.99      0.98      0.99      0.99      0.97     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [84]:
# Resample the training data with the RandomOversampler
ro = RandomOverSampler(random_state=1)
X_ro, y_ro = ro.fit_resample(X_train_ss, y_train)
# View the count of target classes with Counter
Counter(y_ro)

Counter({'low_risk': 56252, 'high_risk': 56252})

In [85]:
# Train the Logistic Regression model using the resampled data
model_re = LogisticRegression(solver='lbfgs', random_state=1)
model_re.fit(X_ro,y_ro)

LogisticRegression(random_state=1)

In [92]:
# Calculated the balanced accuracy score
y_pred_re = model_re.predict(X_test_ss)
balanced_accuracy_score(y_test, y_pred_re)

0.9951657438955139

In [87]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred_re)

array([[  598,     2],
       [  119, 18665]])

In [88]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_re))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



### SMOTE Oversampling

In [90]:
# Resample the training data with SMOTE
smote = SMOTE(random_state=1)
X_smote, y_smote = smote.fit_resample(X_train_ss, y_train)

# View the count of target classes with Counter
Counter(y_smote)

Counter({'low_risk': 56252, 'high_risk': 56252})

In [91]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_smote,y_smote)

LogisticRegression(random_state=1)

In [93]:
# Calculated the balanced accuracy score
y_pred_smote = model_re.predict(X_test_ss)
balanced_accuracy_score(y_test, y_pred_smote)

0.9951657438955139

In [94]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred_smote)

array([[  598,     2],
       [  119, 18665]])

In [95]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      1.00      0.99      0.91      1.00      0.99       600
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18784

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [103]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)

X_cc, y_cc = cc.fit_resample(X_train_ss, y_train)
# View the count of target classes with Counter
Counter(y_cc)

Counter({'high_risk': 1900, 'low_risk': 1900})

In [104]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_cc, y_cc)

LogisticRegression(random_state=1)

In [106]:
# Calculate the balanced accuracy score
y_pred_cc = model_cc.predict(X_test_ss)
balanced_accuracy_score(y_test, y_pred_cc)

0.992878691084611

In [107]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred_cc)

array([[  595,     5],
       [  111, 18673]])

In [108]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [109]:
from imblearn.combine import SMOTEENN 
# Resample the training data with SMOTEENN
smteenn = SMOTEENN(random_state=1)

X_smteenn, y_smteenn = smteenn.fit_resample(X_train_ss, y_train)
# View the count of target classes with Counter
Counter(y_smteenn)

Counter({'high_risk': 55408, 'low_risk': 55899})

In [110]:
# Train the Logistic Regression model using the resampled data
model_smteenn = LogisticRegression(solver='lbfgs', random_state=1)
model_smteenn.fit(X_smteenn, y_smteenn)

LogisticRegression(random_state=1)

In [111]:
# Calculate the balanced accuracy score
y_pred_smteenn = model_smteenn.predict(X_test_ss)
balanced_accuracy_score(y_test, y_pred_smteenn)

0.9951391254968768

In [112]:
# Display the confusion matrix
confusion_matrix(y_test,y_pred_smteenn)

array([[  598,     2],
       [  120, 18664]])

In [113]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       600
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18784

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   SMOTE Oversampling model had the highest accuracy score which makes sense as it is oversampling the data.

2. Which model had the best recall score?

   SMOTEENN model had the best recall score.

3. Which model had the best geometric mean score?

   SMOTE Oversampling model has the best geometric mean score.
